##Librerias necesarias

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import math
import torchtext
from torch.utils.data import DataLoader
from tqdm import tqdm

In [ ]:
!pip install datasets

In [ ]:
import datasets
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Carga de datos y preprocesamiento

In [ ]:
import pandas as pd
import numpy as np
import os
import re
import warnings
warnings.filterwarnings("ignore")
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from bs4 import BeautifulSoup
from nltk.corpus import stopwords

import nltk
nltk.download('stopwords')

import sqlite3
dbfile = '/content/drive/MyDrive/NoticiasReforma2009.db'
import pandas as pd
import sqlite3
import sqlalchemy

try:
    con = sqlite3.connect(dbfile)
except Exception as e:
    print(e)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
cursor = con.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(f"Table Name : {cursor.fetchall()}")


df = pd.read_sql_query('SELECT * FROM Noticias2009', con)
con.close()

Table Name : [('Reforma2009',), ('Noticias2009',)]


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')

stopSpanish=set(stopwords.words('spanish'))
noticias=[]
for i in range (61035):
  text_tokens = word_tokenize(df['articulo'][i])
  tokens_without_sw = [word for word in text_tokens if not word in stopSpanish]
  tokens_without_sw = [word.lower() for word in tokens_without_sw if word.isalpha()]
  noticias.append(' '.join(tokens_without_sw))

stopSpanish=set(stopwords.words('spanish'))
resumen=[]
for i in range (len(df['resumen'])):
  text_tokens = word_tokenize(df['resumen'][i])
  tokens_without_sw = [word for word in text_tokens if not word in stopSpanish]
  tokens_without_sw = [word.lower() for word in tokens_without_sw if word.isalpha()]
  resumen.append(' '.join(tokens_without_sw))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


##Conjuntos de datos final y particiones

In [ ]:
data=pd.DataFrame({'Noticia': noticias, 'Resumen Original': df['resumen'][:], 'Resumen Procesado': resumen})

In [ ]:
train, test = train_test_split(data, test_size=0.1, random_state=2022)
train, validation = train_test_split(train, test_size=0.1, random_state=2022)

##Convertimos a Datasets

In [ ]:
from datasets import Dataset
train_dataset=Dataset.from_pandas(train)
test_dataset=Dataset.from_pandas(test)
validation_dataset=Dataset.from_pandas(validation)

In [ ]:
import datasets
dd = datasets.DatasetDict({"train":train_dataset, 'validation': validation_dataset ,"test":test_dataset})

In [ ]:
dd

DatasetDict({
    train: Dataset({
        features: ['Noticia', 'Resumen Original', 'Resumen Procesado', '__index_level_0__'],
        num_rows: 49437
    })
    validation: Dataset({
        features: ['Noticia', 'Resumen Original', 'Resumen Procesado', '__index_level_0__'],
        num_rows: 5494
    })
    test: Dataset({
        features: ['Noticia', 'Resumen Original', 'Resumen Procesado', '__index_level_0__'],
        num_rows: 6104
    })
})

In [ ]:
dd['train'][0]

{'Noticia': 'solicitan llevar méxico mercosurlegisladores argentinos pidieron pares mexicanos promover participación méxico mercado común sur mercosur pertenecen argentina brasil uruguay encuentro diputados senadores mexicanos vicepresidenta cámara diputados argentina patricia vaca aseguró necesario méxico juegue papel activo latinoamérica forma lograr integración mercosur deseamos méxico cada vez protagonista latinoamérica ver sur existe necesidad protagonista mercosur forma afirmaron legisladores argentinos podría luchar intensamente sucesos golpe estado manera respuesta senadora yeidckol polevnsky afirmó prd congreso siempre demandando verdadera integración méxico américa latina pues prioridad sido estados unidos ello dijo legisladores región deben impulsar creación parlamento latinoamericano semejanza europeo fin asumir mayores compromisos méxico hecho necesario verdadera integración américa latina haciendo cambios suficientes consideró próximos días enviará parlamentos exhorto rea

#Algunos ejemplos para ver que todo bien

In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(dd["train"])

,Noticia,Resumen Original,Resumen Procesado,__index_level_0__
0,prioriza fch instituciones crimenpara derrotar crimen transnacional imprescindible impulsar fortalecimiento instituciones democráticas planteó mañana presidente felipe calderón reunido hotel capital presidente colombia álvaro uribe todavía electo panamá empresario ricardo martinelli calderón propuso dar cuerpo frente regional continental grupos criminales el presidente felipe calderón destacó importancia méxico otorga colaboración regional lucha bajo lógica corresponsabilidad reiteró américa latina oportunidad histórica derrotar crimen trasnacional intenta amenazar desarrollo naciones para ello imprescindible fortalecer instituciones democráticas mejorar leyes hacer frente regional continental criminales además cerrar espacios impunidad informó través extendió uribe martinelli horas éste asumiera cargo presidente invitación participar próxima cumbre regional sobre problema drogas seguridad cooperación celebrará méxico,"En su gira por Panamá, Calderón dijo que para derrotar al crimen transnacional se deben fortaler las instituciones democráticas.",en gira panamá calderón dijo derrotar crimen transnacional deben fortaler instituciones democráticas,40416
1,expropia gobierno local predio inviel gobierno capitalino expropió favor instituto vivienda predio metros cuadrados ubica avenida congreso unión número colonia requisa lote ubicado delegación venustiano carranza publicó martes gaceta oficial destinado edificación vivienda interés social beneficio instituto vivienda pagará indemnización constitucional propietarios resulten afectados expropiación acrediten legítimo derecho consejería monto pagar determinado base avalúo emita dirección general patrimonio inmobiliario,"Un lote de 517 metros que se ubica en la Delegación Venustiano Carranza y será otorgado al Instituto de Vivienda, fue requisado por el GDF.",un lote metros ubica delegación venustiano carranza otorgado instituto vivienda requisado gdf,25320
2,cae carpa festival eslovaquiafuertes vientos provocaron caída carpa festival musical norte eslovaquia causó muerte persona hirió decenas informó ministerio salud lintner director hospital trencin señaló personas resultaron heridas incluidas tres graves lesiones cabeza fue tan repentino había gente cubierta sangre tendida suelo fue horrible relató lucia madarasova años encontraba prensa dijeron cientos personas encontraban interior carpa desplomó festival asistieron mil espectadores,"Durante un festival musical en Eslovaquia, una carpa se desplomó dejando un muerto y decenas de personas heridas.",durante festival musical eslovaquia carpa desplomó dejando muerto decenas personas heridas,45921
3,lleva ochoa cinco años primerauna mañana febrero habitual guardameta titular américa adolfo ríos llegó tarde entrenamiento extrañó comportamiento inusual esperó unas horas escuchó decir condiciones jugar concentró aparente suplente édgar hernández dio oportunidad grupo jovencito larga rizada cabellera creyó iría unas horas partido cumplió sueño el sonido local anunció nombre sorprendió en portería américa guillermo ochoa me acuerdo leo beenhakker buscó mañana concentrado dijo siempre deseado futbolista gustaría hacer aquí y respondí deseaba jugar algún día américa se quedó viendo dijo jugar día llegó tranquilo harás bien vas tener apoyo grupo así tarde jugando experiencia siempre quise estadio azteca américa luego sentí nervios emocioné dijo beenhakker gran apoyo compañeros sentí confianza partido recuerda guillermo guardó ochoa adelantó nadie familia conocidos explotó júbilo haciendo llamadas avisando viera tv simplemente esperó hora destino cumplió no quise decir nadie creí así bien como siempre familia ver partido américa estadio así acostumbraban aunque jugara entonces dejé diera hora partido ahí aparecí sorpresa buena marco aficionados prensa resto componentes américa monterrey quizá rival anhelado ochoa cuando imaginaba día pensaba gustaría estadio azteca lleno partido im

#Carga de las metricas

In [ ]:
!pip install rouge_score

In [ ]:
from datasets import load_dataset, load_metric
metric = load_metric("rouge")
metric

In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 31.4 MB/s 


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


#Instalacion de librerias para modelo

In [ ]:
!pip install transformers

#Cargamos el modelo base para fine tuning

In [ ]:
from transformers import AutoTokenizer
import torch
from transformers import BertTokenizerFast, EncoderDecoderModel
device = 'cuda' if torch.cuda.is_available() else 'cpu'
ckpt = 'mrm8488/bert2bert_shared-spanish-finetuned-summarization'
tokenizer = BertTokenizerFast.from_pretrained(ckpt)
model = EncoderDecoderModel.from_pretrained(ckpt).to(device)

Downloading:   0%|          | 0.00/520 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/242k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.51k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/556M [00:00<?, ?B/s]

The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']


In [ ]:
model_checkpoint='mrm8488/bert2bert_shared-spanish-finetuned-summarization'

#Tenemos que especificar cual es la tarea a realizar

In [ ]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

#Funcion para preprocesar los textos

In [ ]:
max_input_length = 512
max_target_length = 128

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["Noticia"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["Resumen Procesado"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = dd.map(preprocess_function, batched=True)

  0%|          | 0/50 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/7 [00:00<?, ?ba/s]

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']


#Fijamos todos los hiperparametros para el Fine-Tuning

In [ ]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-xsum",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

#Ultimo formato a los datos con los size adecuados

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

#Calculo de las metricas dentro de la evaluacion del modelo

In [ ]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

#Pasamos los argumentos al trainer del modelo

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

#Entrenamiento

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `EncoderDecoderModel.forward` and have been ignored: __index_level_0__, Noticia, Resumen Original, token_type_ids, Resumen Procesado. If __index_level_0__, Noticia, Resumen Original, token_type_ids, Resumen Procesado are not expected by `EncoderDecoderModel.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 49437
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 6180
  Number of trainable parameters = 138840858
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,2.596900,2.455905,49.428200,35.270500,46.095000,46.099400,22.542200
2,2.331800,2.368956,50.020000,35.706000,46.625300,46.641200,22.117600


Saving model checkpoint to bert2bert_shared-spanish-finetuned-summarization-finetuned-xsum/checkpoint-500
Configuration saved in bert2bert_shared-spanish-finetuned-summarization-finetuned-xsum/checkpoint-500/config.json
Model weights saved in bert2bert_shared-spanish-finetuned-summarization-finetuned-xsum/checkpoint-500/pytorch_model.bin
tokenizer config file saved in bert2bert_shared-spanish-finetuned-summarization-finetuned-xsum/checkpoint-500/tokenizer_config.json
Special tokens file saved in bert2bert_shared-spanish-finetuned-summarization-finetuned-xsum/checkpoint-500/special_tokens_map.json
tokenizer config file saved in bert2bert_shared-spanish-finetuned-summarization-finetuned-xsum/tokenizer_config.json
Special tokens file saved in bert2bert_shared-spanish-finetuned-summarization-finetuned-xsum/special_tokens_map.json
Saving model checkpoint to bert2bert_shared-spanish-finetuned-summarization-finetuned-xsum/checkpoint-1000
Configuration saved in bert2bert_shared-spanish-finetun

TrainOutput(global_step=6180, training_loss=2.594882671269784, metrics={'train_runtime': 6658.3145, 'train_samples_per_second': 14.85, 'train_steps_per_second': 0.928, 'total_flos': 3.171422268107352e+16, 'train_loss': 2.594882671269784, 'epoch': 2.0})

#Actualizamos en Huggin Face

In [ ]:
trainer.push_to_hub()

#Ejemplo de uso

In [ ]:
import torch
from transformers import BertTokenizerFast, EncoderDecoderModel
device = 'cuda' if torch.cuda.is_available() else 'cpu'
from transformers import AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained("EP9/bert2bert_shared-spanish-finetuned-summarization-finetuned-xsum")
model.to(device)
tokenizer = BertTokenizerFast.from_pretrained("EP9/bert2bert_shared-spanish-finetuned-summarization-finetuned-xsum")
def generate_summary(text):

   inputs = tokenizer([text], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
   input_ids = inputs.input_ids.to(device)
   attention_mask = inputs.attention_mask.to(device)
   output = model.generate(input_ids, attention_mask=attention_mask)
   return tokenizer.decode(output[0], skip_special_tokens=True)

text = "Your_Text"
generate_summary(text)